This notebook will plot a 3-body problem and calculate the energy of the system over time. 

In [23]:
# Do necessary imports
import numpy as np
from matplotlib import pyplot as plt

In [24]:
# For the Earth, Sun, Moon 3-body Problem:

# Masses of the Planets
Me = 6e24                     # Mass of Earth in kg
Ms = 2e30                     # Mass of Sun in kg                       
Mm = 7.347e22                 # Mass of Moon in kg

# Initial Positions of the Planets (vectors)
Re = np.array([1.496e11*1,0,0]) #1 AU away from the sun converted to m
Rs = np.array([0,0,0])
Rm = np.array([1.496e11*5.2,0,0]) #5.2 AU away from the sun converted to m

# Initial Velocity of the Planets (vectors)
# currently random numbers to see if code runs
Ve = np.array([0,30e3,0])
Vs = np.array([0,0,0])
Vm = np.array([0,10e3,0])


u_0 = np.array([Re[0], Re[1], Re[2],
                Rs[0], Rs[1], Rs[2],
                Rm[0], Rm[1], Rm[2],
                Ve[0], Ve[1], Ve[2],
                Vs[0], Vs[1], Vs[2],
                Vm[0], Vm[1], Vm[2]])

G = 6.673e-11                 # Gravitational Constant

In [25]:
# Calculate the gravitional force on a body given the position vectors and the masses of the bodies
# ====================================================================================================================

def Gravitational_Force(ri, rj, mi, mj, G=6.673e-11):
    return ((G*mi*mj*(np.array(rj)-np.array(ri)))/(np.linalg.norm(np.array(rj)-np.array(ri)))**3)

def force(planet, Re, Rs, Rm):
    force_sun_on_earth = Gravitational_Force(Re,Rs,Me,Ms)
    force_earth_on_sun = -force_sun_on_earth
    force_moon_on_earth = Gravitational_Force(Re,Rm,Me,Mm)
    force_earth_on_moon = -force_moon_on_earth
    force_sun_on_moon = Gravitational_Force(Rm,Rs,Mm,Ms)
    force_moon_on_sun = -force_sun_on_moon
    
    if planet == "earth":
        return force_sun_on_earth + force_moon_on_earth
    if planet == "sun":
        return force_earth_on_sun + force_moon_on_sun
    if planet == "moon":
        return force_sun_on_moon + force_earth_on_moon
    
def acc_planet(planet,Re,Rs,Rm,M):
    a = force(planet, Re, Rs, Rm)/M
    return a
    
def f_true(u):
    Re = [u[0], u[1], u[2]]
    Rs = [u[3], u[4], u[5]]
    Rm = [u[6], u[7], u[8]]

    a_earth = acc_planet('earth',Re,Rs,Rm,Me)
    a_sun = acc_planet('sun',Re,Rs,Rm,Ms)
    a_moon = acc_planet('moon',Re,Rs,Rm,Mm)
    

    udot = np.array([u[9], u[10], u[11],
                     u[12], u[13], u[14],
                     u[15], u[16], u[17],
                     a_earth[0], a_earth[1], a_earth[2],
                     a_sun[0], a_sun[1], a_sun[2], 
                     a_moon[0], a_moon[1], a_moon[2]])
    
    return udot

In [26]:
# IVP Forward Euler Code
import numpy as np

def ivp_forward_euler(u_0, T, delta_t):
    """Returns the predicted system evolution over time using the forward Euler method.

    This function assumes an f_true(u) function is globally available for calculating the true function value at u.
    
    Parameters
    ----------
    u_0 : array
        1 x N array defining the initial state vector u_0 where N = number of state variables = |u|
    T : float_like
        Final time T
    delta_t : float_like
        Time step size where delta_t = t_{k+1} - t_k
        
    Returns
    -------
    u : array
        K x N array of the predicted states where row k is the state at time step k and K = number of time steps
    times : array_like
        Length K vector containing the times t corresponding to time steps
    """
    

    times = np.arange(0, T+delta_t,delta_t)
    u = np.zeros((len(times),len(u_0)))
    for i in range(len(times)):
        if i == 0:
            u[0] = u_0
        else:  
            u[i] = u[i-1] + delta_t*f_true(u[i-1])
    
    return u, times
    

In [27]:
# Functions to calculate energy of the system 
# ==================================================================================================================

# Calculate the kinetic energy
def Kinetic_Energy(m,v):
    v_n = np.linalg.norm(v)
    return 0.5*m*v_n**2

# Calculate the magnitude of the potential energy to be used when calculating the total energy
def PE_mag(mi,mj,ri,rj,G=6.673e-11):
    return G*mi*mj/np.linalg.norm(ri-rj)

# Calculate the total energy for the 3-body problem
def Total_Energy(m1,m2,m3,r1,r2,r3,v1,v2,v3):
    total_KE = Kinetic_Energy(m1,v1)+Kinetic_Energy(m2,v2)+Kinetic_Energy(m3,v3)
    total_PE = -(PE_mag(m2,m1,r2,r1)+PE_mag(m3,m1,r3,r1)+PE_mag(m3,m2,r3,r2))
    return total_KE + total_PE

In [28]:
ivp_forward_euler(u_0, 9.5e7, 86400)

(array([[ 1.49600000e+11,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  1.00000000e+04,  0.00000000e+00],
        [ 1.49600000e+11,  2.59200000e+09,  0.00000000e+00, ...,
         -1.90544749e+01,  1.00000000e+04,  0.00000000e+00],
        [ 1.49555484e+11,  5.18400000e+09,  0.00000000e+00, ...,
         -3.81089146e+01,  9.99997884e+03,  0.00000000e+00],
        ...,
        [ 6.64451246e+10,  1.97273750e+11,  0.00000000e+00, ...,
         -1.51009564e+04, -1.52225440e+04,  0.00000000e+00],
        [ 6.43471696e+10,  1.98019351e+11,  0.00000000e+00, ...,
         -1.50631002e+04, -1.52923774e+04,  0.00000000e+00],
        [ 6.22418749e+10,  1.98743162e+11,  0.00000000e+00, ...,
         -1.50248123e+04, -1.53622278e+04,  0.00000000e+00]]),
 array([0.00000e+00, 8.64000e+04, 1.72800e+05, ..., 9.48672e+07,
        9.49536e+07, 9.50400e+07]))

In [10]:
np.array([1,2,3])-np.array([1,2,3])

array([0, 0, 0])